In [1]:
import numpy as np
import pandas as pd
import os
import csv

In [2]:
# import data
logfile = open("../../../Data/Logs/light1_first_se_log.txt")
# logfile = open("../../Data/Logs/Light1_OperationalTest_May6_ScriptLog_1.txt")
loglines = logfile.readlines()
print (loglines)

['# Script started\n', 'SE0>rtc read\n', 'curr rtc time 1646297867.353394 s, 2022-03-03 08:57:47 \n', 'OK\n', 'SE0># Script delay (avoiding SAA)\n', 'SE0>script delayuntil 1646297935\n', 'OK\n', 'SE0># Enabling the PC and payload (and switching off with the timer of 30m for the payload and 3h for the PC)\n', 'SE0>csp txrx 4 14 600 000500302A\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0008000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0005010100\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007011400\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp t

In [3]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    if "SE0>rtc read" in k:
        I = i+1
        l = loglines[I][14:]
        print ("Line:" , i , ", Command:" , k , ", Time:" , l )

# if the value is in range of the full range of file?
# first rtc must be start of code, second rtc must be end of code

Line: 1 , Command: SE0>rtc read
 , Time: 1646297867.353394 s, 2022-03-03 08:57:47 

Line: 139 , Command: SE0>rtc read
 , Time: 1646299664.765045 s, 2022-03-03 09:27:44 



In [6]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    # l = loglines[i+1]
    if k[0:12] in "SE0>rtc read":
        # l=i+1
        print ("line: " , i , ", command: " , k )




    # if k[0:5] in "SE0>#":
    #     print (k)

line:  1 , command:  SE0>rtc read

line:  139 , command:  SE0>rtc read

line:  142 , command:  SE0>


In [7]:
# Work to find the following:

# "Time,scenario"
# "At time () command () was excuted.
# "At time () command () failed."

# If line in SE0>:
# print the time
# print the command
# print the info
# print if command was excuted or failed

In [144]:
def desc_finder(line:str,cmdlist,outputs,i,time,failed_idx):

    # print(line)
    status = 1
    splt = line.split(' ')

    # print (splt)

    if splt[-1] == 'SE0>':
        desc = 'LOG END'

    elif 'txrx' in splt[1]:
        node,port,msg = int(splt[2]),int(splt[3]),str(splt[5])
        index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port) & (cmdlist['Message'].str.startswith(msg)),['ID COMMAND (Proposed)']]
        
        # include the power shutdown
        if node == 4:
            index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port),['ID COMMAND (Proposed)']]
        

        #If index did not find anything
        if len(index) == 0:
            # desc = ('Not Payload Command: '+ line)
            desc = ''
            
        # if command found in command list
        else:
            c = index.to_string(index=False).split('\n')[1].strip()
            desc = c
        

        if i in [fid for fid in failed_idx]:
            time = time + (float(splt[4])/1000)
            status = -1
        

    elif splt[1] == 'delay':
        # desc = 'Delay of: ' + splt[2] + 'ms'
        # time = time + (float(splt[2])/1000)
        desc = ''

    elif splt[1] == 'delayuntil':
        # desc = 'Delayuntil: ' + splt[2] + 's'
        time = float(splt[2])
        desc = ''

    elif 'read' in splt[1]:
        t = outputs[i][0].split(' ')[3:-1]
        # desc = 'Read the Current Time: ' + ' '.join([str(elem) for elem in t])
        time = float(outputs[i][0].split(' ')[3])
        desc = ''

    else:
        # desc = 'Different Command Type: ' + line
        desc = ''
    
    return desc,time,status#,scenario

In [145]:
def decode_log(filename:str="../../../Data/Logs/light1-SD-1016-se-log.txt"):

    # Load the logfile
    logfile = open(filename)
    cmdlist = pd.read_csv("command_list.csv")

    # Load the lines
    loglines = logfile.readlines()

    # Close the file
    logfile.close()

    # Create an array with the lines
    commands    = []
    outputs     = []
    idx         = []
    description = []
    times       = []
    fails       = []
    failed_idx = []

    # Get commands and their indices
    for i, line in enumerate(loglines):
        if 'SE0>' in line:
            commands.append(line)
            idx.append(i)

    # Check if the last command was empty
    if loglines[-1] != 'SE0>':
        commands.append('SE0>')
        idx.append(len(loglines))

    # Get the command output
    for i in range(len(idx)-1):
        out = []
        if 'SE0>#' not in commands[i]:
            for j in range(idx[i]+1,idx[i+1]):
                out.append(loglines[j])
        
        outputs.append(out)

    # cmdid = dict(zip(idx, commands))

    # Create the dictionary
    log = [[command,output] for command,output in zip(commands,outputs)]

    # Check which commands executed correcly
  

    # Find the commands that failed
    for i in range(len(commands)-1):
        for output in log[i][1]:
            if 'FAIL' in output:
                failed_idx.append(i)


    # Initialize time at 0s
    time = 0
    
    # Give commands and use the finder function to pull the description, time ran, and if the command failed.
    for k, id in enumerate(idx):
        c = commands[k]
        cmd = c.split('\n')[0]
        desc,time,failed = desc_finder(cmd,cmdlist,outputs,k,time,failed_idx)

        if not desc == '':
            description.append(desc)
            times.append(time)
            fails.append(failed)

    #Name the CSV file and create it if it doesn't exist
    tempname = 'templog.csv'
    fileDir = os.path.join('Defined_Logs')

    if not os.path.exists(fileDir):
        os.makedirs(fileDir)
    
    filePath = os.path.join(fileDir, tempname)

    #Name the headers of the CSV File
    header = ['status,time,description']

    #Clear the file and write from scratch
    with open(filePath, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

        for i in range(len(description)):
            g = (fails[i], times[i-1], description[i])
            writer.writerow(g)


    # Create a dictionary
    return log, commands[:-1], outputs, description, failed_idx

In [146]:
# Get the log file and the commands
log, commands, outputs, description, failed_idx   = decode_log()

In [50]:
failed_idx

[23,
 25,
 30,
 57,
 99,
 101,
 106,
 175,
 177,
 209,
 283,
 326,
 327,
 328,
 329,
 334,
 402,
 403,
 405,
 410,
 414,
 423,
 478,
 479,
 481,
 555,
 557,
 589,
 633,
 640,
 665,
 667,
 708,
 711,
 716,
 784,
 785,
 792,
 819,
 861,
 863,
 868,
 893,
 895,
 944,
 969]

In [27]:
# Check which commands executed correcly
failed_idx = []

# Find the commands that failed
for i in range(len(commands)):
    for output in log[i][1]:
        if 'FAIL' in output:
            failed_idx.append(i)

print(failed_idx)

NameError: name 'commands' is not defined

In [54]:
# print (failed_idx)
# print (failed_cmd,failed_output)
# print (failed_output)

In [ ]:
# Decode the file

In [ ]:
# Define the command description in a dict
# return timestamps of the scenarios and the failures (use the delays and the rtc reads to find the time)
# id,line,command,description,output(ok,or fail),timetaken


In [ ]:
# line = SE0>csp txrx 12 8 3000 08
def f(line:str):
    line[13:15]